In [1]:
import torch 
import random
import numpy as np
import os
from torchvision.datasets import ImageFolder
from PIL import Image
from efficientnet_pytorch import EfficientNet

In [2]:
# 훈련, 검증, 테스트 데이터 경로 설정
train_path = './Dataset/dct_train'
valid_path = './Dataset/dct_valid'
test_path = './Dataset/dct_test'
n_patch = 5

research_name = 'Resarch3_3'

In [3]:
# 시드값 고정

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [4]:
#GPU 장비 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from torchvision.transforms import functional as F
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import scipy.fftpack
    
transform = transforms.Compose([
    transforms.ToTensor()
])

# 훈련 데이터셋
datasets_train = ImageFolder(train_path, transform=transform)
# 검증 데이터셋
datasets_valid = ImageFolder(valid_path, transform=transform)

In [6]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 제너레이터 시드값 고정
g = torch.Generator()
g.manual_seed(0)

In [7]:
from torch.utils.data import DataLoader

batch_size = 64

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=0)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=0)

In [8]:

# 모델 생성
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2) 
# 장비 할당
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [9]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
import wandb

# wandb 초기화: 새로운 실험을 생성하거나 기존 실험에 연결
wandb.init(project="FakeImageDetection_Research", name=research_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: guddnjs2366 (seohw). Use `wandb login --relogin` to force relogin


In [12]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score       # F1 점수 계산 함수
from tqdm.notebook import tqdm             # 진행률 표시 막대

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=10, save_file=''):
    
    valid_loss_min = np.inf # 최소 손실값 초기화 (검증 데이터용) 

    # 총 에폭만큼 반복
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
        
        # == [ 훈련 ] ==============================================
        model.train()        # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
        train_preds_list = []  # 훈련 데이터 예측값 저장용 리스트 초기화
        train_true_list = []   # 훈련 데이터 실젯값 저장용 리스트 초기화
        # '반복 횟수'만큼 반복 
        for images, labels in tqdm(loader_train):
            # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당 
            images = images.to(device)
            labels = labels.to(device)
            
            # 옵티마이저 내 기울기 초기화
            optimizer.zero_grad()
            # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
            outputs = model(images)
            # 손실 함수를 활용해 outputs와 labels의 손실값 계산
            loss = criterion(outputs, labels)
            # 현재 배치에서의 손실 추가 (훈련 데이터용)
            epoch_train_loss += loss.item() 
            loss.backward()       # 역전파 수행
            optimizer.step()      # 가중치 갱신
            if scheduler != None: # 스케줄러 학습률 갱신 
                scheduler.step() 
                
            # 예측값 및 실제값 저장 (훈련 데이터용)
            train_preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
            train_true = labels.cpu().numpy()
            train_preds_list.extend(train_preds)
            train_true_list.extend(train_true)
            
        # 훈련 데이터 정확도 계산
        train_accuracy = accuracy_score(train_true_list, train_preds_list)

        # 훈련 데이터 손실값 출력
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        print(f'\t훈련 데이터 정확도 : {train_accuracy:.4f}')
        
        # == [ 검증 ] ==============================================
        model.eval()         # 모델을 평가 상태로 설정 
        epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
        preds_list = []      # 예측값 저장용 리스트 초기화
        true_list = []       # 실젯값 저장용 리스트 초기화
        
        with torch.no_grad(): # 기울기 계산 비활성화
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실제값 
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
                true = labels.cpu().numpy() 
    
                preds_list.extend(preds)
                true_list.extend(true)
                
        # 정확도, 재현율, F1 점수 계산
        val_accuracy = accuracy_score(true_list, preds_list)
        val_recall = recall_score(true_list, preds_list)
        val_f1_score = f1_score(true_list, preds_list)

        wandb.log({"train_loss": epoch_train_loss/len(loader_train),
                   "train_accuracy" : train_accuracy,
                   "val_loss": epoch_valid_loss/len(loader_valid), 
                   "val_accuracy": val_accuracy,
                   "val_recall" : val_recall,
                   "val_f1_score" : val_f1_score})
        
        # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
        print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        # == [ 최적 모델 가중치 찾기 ] ==============================
        # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 
        if epoch_valid_loss <= valid_loss_min: 
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
            # 모델 가중치를 파일로 저장 
            torch.save(model.state_dict(), f'./Predict/{save_file}/'+ save_file + f'_{epoch}epoch.pth') 
            valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 
            high_epoch = epoch
            
        if epoch == 9:
            torch.save(model.state_dict(), f'./Predict/{save_file}/'+ save_file + f'_lastepoch.pth') 
            
    return torch.load(f'./Predict/{save_file}/'+ save_file + f'_{high_epoch}epoch.pth') # 저장한 모델 가중치를 불러와 반환

In [13]:
os.makedirs(f'./Predict/{research_name}', exist_ok=True)
# 모델 훈련
model_state_dict = train(model = model,
                         loader_train = loader_train, 
                         loader_valid = loader_valid,
                         criterion = criterion, 
                         optimizer = optimizer,
                         save_file = research_name)

torch.cuda.empty_cache()
wandb.finish()

에폭 [1/10] 
-----------------------------


  0%|          | 0/947 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0353
	훈련 데이터 정확도 : 0.9872
	검증 데이터 손실값 : 0.0416
	정확도 : 0.9850 / 재현율 : 0.9997 / F1 점수 : 0.9899
	### 검증 데이터 손실값 감소 (inf --> 9.9062). 모델 저장
에폭 [2/10] 
-----------------------------


  0%|          | 0/947 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0097
	훈련 데이터 정확도 : 0.9968
	검증 데이터 손실값 : 0.0576
	정확도 : 0.9790 / 재현율 : 1.0000 / F1 점수 : 0.9859
에폭 [3/10] 
-----------------------------


...

  0%|          | 0/947 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0039
	훈련 데이터 정확도 : 0.9986
	검증 데이터 손실값 : 0.0037
	정확도 : 0.9988 / 재현율 : 0.9996 / F1 점수 : 0.9992
에폭 [10/10] 
-----------------------------


  0%|          | 0/947 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0026
	훈련 데이터 정확도 : 0.9990
	검증 데이터 손실값 : 0.0044
	정확도 : 0.9990 / 재현율 : 1.0000 / F1 점수 : 0.9993


train_accuracy,▁▇▇▇██████
train_loss,█▃▂▃▁▂▁▁▁▁
val_accuracy,▆▆██▇▁████
val_f1_score,▆▆██▇▁████
val_loss,▂▃▁▁▂█▁▁▁▁
val_recall,█████▁████
train_accuracy,0.99901
train_loss,0.00257
val_accuracy,0.99901
val_f1_score,0.99933
val_loss,0.00439


In [14]:
# 최적 가중치 불러오기
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [15]:
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, images_folder, transform=None):
        self.images_folder = images_folder
        self.transform = transform

        # 폴더 내의 이미지 파일 리스트
        self.image_filenames = [os.path.join(self.images_folder, file) for file in os.listdir(self.images_folder)
                                if file.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # 이미지 불러오기
        image_path = self.image_filenames[idx]
        image = Image.open(image_path).convert('RGB')

        # 변환 적용
        if self.transform:
            image = self.transform(image)

        return image

In [16]:
import pandas as pd

for i in range(n_patch):
    print(f'{i+1}_patch start predict')
    
    datasets_test = CustomImageDataset(images_folder=test_path + f'/images_{i}', transform=transform)

    batch_size = 256

    loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, 
                            shuffle=False, worker_init_fn=seed_worker,
                            generator=g, num_workers=0)
    model.eval()  # 평가 모드로 전환

    # 3. 예측하기
    predictions = []

    for image in tqdm(loader_test):
        image = image.to(device)  # 장비 할당
        with torch.no_grad():
            outputs = model(image)
            prediction = outputs.argmax(dim=1).cpu().numpy()
            predictions.extend(prediction)

    torch.cuda.empty_cache()
    
    # 4. 결과 저장
    df = pd.read_csv(r'C:\Users\user\Desktop\Workspace\Thesis\generative_image_detection\Personal Research\test_data\sample_submission.csv')
    df['answer'] = predictions
    df.to_csv(f'./Predict/{research_name}/predict_{i}patch.csv', index=False)
    


1_patch start predict


  0%|          | 0/228 [00:00<?, ?it/s]

2_patch start predict


  0%|          | 0/228 [00:00<?, ?it/s]

3_patch start predict


  0%|          | 0/228 [00:00<?, ?it/s]

4_patch start predict


  0%|          | 0/228 [00:00<?, ?it/s]

5_patch start predict


  0%|          | 0/228 [00:00<?, ?it/s]

In [17]:
from glob import glob 
from collections import Counter

def hard_voting(csv_path, save_path):
    csv_list = glob(f'{csv_path}/*.csv')

    answer_columns = []

    # Load each csv file
    for csv_file in csv_list:
        df = pd.read_csv(csv_file)
        answer_columns.append(df['answer'])

    # Concatenate along the column axis
    combined_answers = pd.concat(answer_columns, axis=1)

    # Apply a lambda function to each row to find the most common value (mode)
    voted_answers = combined_answers.apply(lambda row: Counter(row).most_common(1)[0][0], axis=1)

    # Load first csv file to get 'ImageId' column
    df = pd.read_csv(csv_list[0])
    ImageId = df['ImageId']

    # Create a new DataFrame with 'ImageId' and voted answers
    result_df = pd.DataFrame({'ImageId': ImageId, 'answer': voted_answers})

    # Save the result to a csv file
    result_df.to_csv(f'{save_path}/voted_result.csv', index=False)

In [18]:
hard_voting(f'./Predict/{research_name}', f'./Predict/{research_name}')